# Single Allocation P-hub Location Median Problem using Simulated Annealing

## Importing Package and Dataset

In [1]:
# Importing Package and Libraries

import numpy as np
import pandas as pd
import random
import itertools
import time
from collections import Counter
import math

In [2]:
# Importing Dataset Flow
# This dataset consists of flow from each nodes
CAB_10_nodes_flow = pd.read_csv("10_nodes_CAB_flow.csv", delimiter= ";", header = None)
CAB_25_nodes_flow = pd.read_csv("25_nodes_CAB_flow.csv", delimiter= ";", header = None)
TR_55_nodes_flow = pd.read_csv("55_nodes_TR_flow.csv", delimiter= ";", header = None)
TR_81_nodes_flow = pd.read_csv("81_nodes_TR_flow.csv", delimiter= ";", header = None)
RGP_100_nodes_flow = pd.read_csv("100_nodes_RGP_flow.csv", delimiter= ";", header = None)                                     

In [3]:
CAB_10_nodes_flow

,0,1,2,3,4,5,6,7,8,9
0,0,6469,7629,20036,4690,6194,11688,2243,8857,7248
1,6469,0,12999,13692,3322,5576,3878,3202,6699,4198
2,7629,12999,0,35135,5956,14121,5951,5768,16578,4242
3,20036,13692,35135,0,19094,35119,21423,27342,51341,15826
4,4690,3322,5956,19094,0,7284,3102,1562,7180,1917
5,6194,5576,14121,35119,7284,0,5023,3512,10419,3543
6,11688,3878,5951,21423,3102,5023,0,11557,6479,34261
7,2243,3202,5768,27342,1562,3512,11557,0,5615,7095
8,8857,6699,16578,51341,7180,10419,6479,5615,0,4448
9,7248,4198,4242,15826,1917,3543,34261,7095,4448,0


In [4]:
# Importing Dataset Cost
# This dataset consists of cost from each nodes
CAB_10_nodes_cost = pd.read_csv("10_nodes_CAB_cost.csv", delimiter= ";", header = None)
CAB_25_nodes_cost = pd.read_csv("25_nodes_CAB_cost.csv", delimiter= ";", header = None)
TR_55_nodes_cost = pd.read_csv("55_nodes_TR_cost.csv", delimiter= ";", header = None)
TR_81_nodes_cost = pd.read_csv("81_nodes_TR_cost.csv", delimiter= ";", header = None)
RGP_100_nodes_cost = pd.read_csv("100_nodes_RGP_cost.csv", delimiter= ";", header = None)

In [5]:
CAB_10_nodes_cost

,0,1,2,3,4,5,6,7,8,9
0,0.0000,576.9631,946.4954,597.5972,373.8127,559.7673,709.0215,1208.3280,603.6477,695.2080
1,576.9631,0.0000,369.5327,613.0386,429.1079,312.8831,1196.4890,1502.1400,405.8975,1241.9610
2,946.4954,369.5327,0.0000,858.3308,749.6018,556.0706,1541.2730,1764.7910,621.3306,1603.1650
3,597.5972,613.0386,858.3308,0.0000,255.0303,311.3071,790.1213,907.4331,237.0703,932.2173
4,373.8127,429.1079,749.6018,255.0303,0.0000,225.8954,794.1726,1080.3740,238.9440,879.5647
5,559.7673,312.8831,556.0706,311.3071,225.8954,0.0000,1009.6890,1216.8680,94.2588,1104.5740
6,709.0215,1196.4890,1541.2730,790.1213,794.1726,1009.6890,0.0000,663.8762,982.7378,221.4220
7,1208.3280,1502.1400,1764.7910,907.4331,1080.3740,1216.8680,663.8762,0.0000,1143.7910,874.5181
8,603.6477,405.8975,621.3306,237.0703,238.9440,94.2588,982.7378,1143.7910,0.0000,1094.9060
9,695.2080,1241.9610,1603.1650,932.2173,879.5647,1104.5740,221.4220,874.5181,1094.9060,0.0000


## Random Initial Solution

In [6]:
# Random Initial Solution Function

def random_initial_solution (cost_matrix, number_hubs):
    
    number_nodes = cost_matrix.shape[0] # Number of Nodes is based on Cost Matrix Shape.
    nodes = range(1, number_nodes + 1) # Nodes is range from 1 to total nodes + 1
    
    hubs = random.sample(nodes, number_hubs) # Select the hubs randomly.
    spokes = [node for node in nodes if node not in hubs] # Spokes is node which is not in the hub list defined before.
    initial_solution = [0]*number_nodes # Initial Solution is array format with width total number of nodes. As an initial solution, the array consists of zero value.
    
    for node in nodes:
        if node in hubs: # If node is listed on hub list
            initial_solution [node - 1] = node # Then Initial Solution for Node - 1 equals with Node
        else:
            hub_spoke_cost = {hub : cost_matrix[node - 1][hub - 1] for hub in hubs}
            initial_solution[node - 1] = min(hub_spoke_cost, key = hub_spoke_cost.get)
    
    return initial_solution # Initial Solution for Hub Selection and Assignment

In [7]:
# Test Random Initial Function

test_solution_1 = random_initial_solution (CAB_10_nodes_cost, 3)

print ('Initial Solution Assigment: ', test_solution_1)
print ('Hub: ', set (test_solution_1))

Initial Solution Assigment:  [5, 5, 5, 4, 5, 5, 10, 10, 4, 10]
Hub:  {10, 4, 5}


## Cost Function

In [9]:
# Flow loc function
# Flow loc function is used to get economic of scale based on piece wise

def flow_loc_function (flow):
    if flow < 0:
        raise ValueError ('No negative value')
    elif flow < 50000:
        unit_flow_cost = 0 + 1*flow
    elif flow < 100000:
        unit_flow_cost = 10000 + 0.8*flow
    elif flow < 200000:
        unit_flow_cost = 30000 + 0.6*flow
    elif flow >= 200000:
        unit_flow_cost = 70000 + 0.4*flow

    return unit_flow_cost

In [10]:
# Test Flow Loc Function

test_flow_loc1 = flow_loc_function (10000)

print ('Unit Flow Cost: ', test_flow_loc1)

Unit Flow Cost:  10000


In [11]:
# Total Network cost function

def network_cost (solution, flow_matrix, cost_matrix):
    
    number_nodes = cost_matrix.shape[0]
    cost = 0 # Cost is initialized with zero value

    spoke_to_hub_cost = 0
    for node_1 in range(number_nodes): # For Node 1 in the range of number nodes
        for node_2 in range(number_nodes): # For Node 2 in the range of number nodes
            spoke_to_hub_cost += (flow_matrix[node_1][node_2] * (cost_matrix[node_1][solution[node_1]-1] + cost_matrix[solution[node_2]-1][node_2]))

    hub = set (solution) # Based on solution, check the unique number, which are the hubs number.

    hub_to_hub_cost = 0
    for hub_k in (hub): # Choose Hub K  
        for hub_m in (hub): # Choose Hub M
            if hub_k != hub_m: # Hub K not equals with Hub M
                hub_to_hub_flow = 0
                for node_1 in range (number_nodes): # For all Node 1 in range of number nodes
                    for node_2 in range (number_nodes): # For all Node 2 in range of number nodes
                        if node_1 != node_2:
                            if (solution[node_1] == hub_k) and (solution[node_2]) == hub_m:
                                hub_to_hub_flow += flow_matrix[node_1][node_2]

                hub_to_hub_cost += flow_loc_function (hub_to_hub_flow) * cost_matrix[hub_k-1][hub_m-1]
    
    cost = spoke_to_hub_cost + hub_to_hub_cost

    return cost

In [12]:
# Test Cost Network

test_cost1 = network_cost ([4, 9, 9, 4, 4, 9, 7, 4, 9, 7], CAB_10_nodes_flow, CAB_10_nodes_cost)

print ('Total Cost Network: ', test_cost1)

Total Cost Network:  740999438.9360802


## Neighborhood Structure (NS)

In Simulated Annealing, there is a need as well to define neighborhood structure (NS). The neighborhood is needed to produce new candidate solution. In order to produce a good solution, it is necessary to create randomness between hub selection and assignment process.

In Neighborhood structure, they will use initial solution which is already defined.

In [13]:
# Neighbourhood Structure 1: Reallocate Hub - Swap Hub (Random Hub to Non Hub (Spoke), and Random Non Hub (Spoke) to Hub)
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Efficient Algorithm for the uncapacitated single allocation p-hub median problem - 1996
    # Paper 3: Efficient Simulated Annealing based solution approaches to the competitive single and multiple allocation hub location problem

def NS1 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hub_to_spoke = random.choice (list(set(solution))) # Selecting randomly from solution list (hub list) to be a spoke.

    spokes = [node for node in range(1, node+1) if node not in set(solution)] # Spoke is in the range 1 to node+1 if it is not in solution list (hub list).

    spoke_to_hub = random.choice (spokes) # Selecting randomly spokes (non hub) from spoke list (spokes) to be a hub.   

    solution = [spoke_to_hub if x == hub_to_spoke else x for x in solution] # Solution is  new hub (from spoke) if x equals with new spoke (from hub), else it is x for x in solution.

    solution[spoke_to_hub-1] = spoke_to_hub # Solution (Spoke to hub - 1) equals with new hub (from spoke).

    return solution

In [14]:
# Neighbourhood Structure 1

solution_before_NS1 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

solution_after_NS1 = NS1 (solution_before_NS1, CAB_25_nodes_cost)

print ('Solution Before NS1', solution_before_NS1 )
print ('Solution After NS1', solution_after_NS1 )

Solution Before NS1 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution After NS1 [21, 21, 22, 6, 21, 6, 6, 6, 6, 6, 6, 6, 22, 21, 6, 6, 21, 21, 22, 21, 21, 22, 6, 21, 21]


In [15]:
# Neighbourhood Structure 2: Swap Nodes: Swap the allocations of two (randomly chosen) non hub nodes from different cluster.
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Mutation Procedure - Solving the uncapacitated hub location problem using genetic algorithm - 2005
    # Paper 3: Using simulated annealing to solve the p-hub median problem

def NS2 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution)) # Hubs is based on list of solution.

    hubs_chosen = random.sample (hubs, 2) # Select randomly based on list of solution. 2 is number of items to be returned.

    hub_spoke  = {}
    for hub in set(solution):
        hub_spoke[hub] = [i+1 for i,val in enumerate(solution) if val==hub]
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes.

    l1 = hub_spoke[hubs_chosen[0]] # l1 is hub chosen first.
    l2 = hub_spoke[hubs_chosen[1]] # l2 is hub chosed second.
    
    spoke1 = random.choice(l1)
    spoke2 = random.choice(l2)

    solution[spoke1-1] = hubs_chosen[1]
    solution[spoke2-1] = hubs_chosen[0]
    
    solution[hubs_chosen[0]-1] = hubs_chosen[0]
    solution[hubs_chosen[1]-1] = hubs_chosen[1]
    
    return solution

In [16]:
# Test Neighbourhood Structure 2

solution_before_NS2 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

print ('Solution before NS2', solution_before_NS2)

solution_after_NS2 = NS2 (solution_before_NS2, CAB_25_nodes_cost)

print ('Solution after NS2', solution_after_NS2)

Solution before NS2 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution after NS2 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 21, 21, 21, 6, 6, 21, 3]


In [17]:
# Neighbourhood Structure 3: Swapping hubs - Swaping the nodes of two hubs

def NS3 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution))
    hubs_chosen = random.sample(hubs, 2)

    hub_spoke = {}
    for hub in set(solution):
        hub_spoke[hub] = [i for i,val in enumerate(solution) if val==hub]#it takes the idexes of the hubs in the solution
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes 
    
    for i in hub_spoke[hubs_chosen[0]]:
        solution[i] = hubs_chosen[1]
    for i in hub_spoke[hubs_chosen[1]]:
        solution[i] = hubs_chosen[0]
    
    return solution

In [18]:
# Test Neighbourhood Structure 3

solution_before_NS3 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

print ('Solution before NS3', solution_before_NS3)

solution_after_NS3 = NS3 (solution_before_NS3, CAB_25_nodes_cost)

print ('Solution after NS3', solution_after_NS3)

Solution before NS3 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution after NS3 [3, 3, 3, 6, 3, 6, 6, 6, 6, 6, 6, 6, 21, 3, 6, 6, 3, 3, 21, 3, 21, 6, 6, 3, 3]


In [19]:
# Neighbourhood Structure 4: Reallocate a node - Change the allocation of a randomly chosen non hub node to a different randomly chosen hub
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Efficient Algorithm for the uncapacitated single allocation p-hub median problem - 1996
    # Paper 3: Efficient Simulated Annealing based solution approaches to the competitive single and multiple allocation hub location problem
    # Paper 4: Mutation Procedure - Solving the uncapacitated hub location problem using genetic algorithm - 2005

def NS4 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution))
    
    nodes = list(range(1,node+1))
    
    for hub in hubs:
        nodes.remove(hub)

    spoke = random.choice(nodes)

    hub1 = solution[spoke-1]
    hubs.remove(hub1)
    hub2 = random.choice(hubs)

    solution[spoke-1] = hub2
    
    return solution

In [20]:
solution_before_NS4 = [1, 1, 11, 69, 6, 6, 35, 69, 35, 35, 11, 44, 35, 6, 11, 11, 44, 6, 6, 35, 44, 11, 44, 69, 69, 11, 1, 6, 69, 44, 1, 11, 1, 34, 35, 69, 6, 6, 11, 6, 11, 6, 11, 44, 35, 1, 44, 35, 44, 6, 1, 1, 69, 11, 6, 44, 6, 6, 11, 6, 69, 69, 1, 35, 44, 6, 6, 6, 69, 1, 6, 44, 44, 6, 69, 69, 11, 6, 44, 44, 34]

print ('Solution before NS4', solution_before_NS4)

solution_after_NS4 = NS4 (solution_before_NS4, TR_81_nodes_cost)

print ('Solution after NS4', solution_after_NS4)

Solution before NS4 [1, 1, 11, 69, 6, 6, 35, 69, 35, 35, 11, 44, 35, 6, 11, 11, 44, 6, 6, 35, 44, 11, 44, 69, 69, 11, 1, 6, 69, 44, 1, 11, 1, 34, 35, 69, 6, 6, 11, 6, 11, 6, 11, 44, 35, 1, 44, 35, 44, 6, 1, 1, 69, 11, 6, 44, 6, 6, 11, 6, 69, 69, 1, 35, 44, 6, 6, 6, 69, 1, 6, 44, 44, 6, 69, 69, 11, 6, 44, 44, 34]
Solution after NS4 [1, 1, 11, 69, 6, 6, 35, 69, 35, 35, 11, 44, 35, 6, 11, 11, 44, 6, 6, 35, 44, 11, 44, 69, 69, 11, 1, 6, 69, 44, 1, 11, 1, 34, 35, 69, 6, 6, 11, 6, 11, 6, 11, 44, 35, 1, 44, 35, 44, 6, 1, 1, 69, 11, 6, 44, 6, 44, 11, 6, 69, 69, 1, 35, 44, 6, 6, 6, 69, 1, 6, 44, 44, 6, 69, 69, 11, 6, 44, 44, 34]


In [21]:
# Neighbourhood Structure 5: Local Search using Reallocate Node

def NS5 (solution, flow_matrix, cost_matrix):
    
    # Initial Solution
    initial_solution = solution
    initial_solution_cost = network_cost (solution, flow_matrix, cost_matrix)
    hubs = list(set(initial_solution))
    
    # Best Solution
    best_solution = initial_solution.copy()
    best_cost = initial_solution_cost
    
    # Best Neighbourhood
    best_solution_neighbourhood = None
    best_cost_neighbourhood = float ('inf')

    for node in range (1, len (initial_solution)+1):
        if node in hubs:
            continue
        for new_hub in hubs:
            if new_hub == initial_solution[node-1]:
                continue
            else:
                neighbour_solution_result = initial_solution.copy()
                neighbour_solution_result[node-1] = new_hub
                neighbour_cost = network_cost (neighbour_solution_result, flow_matrix, cost_matrix)

            if neighbour_cost < best_cost_neighbourhood:
                best_cost_neighbourhood = neighbour_cost
                best_solution_neighbourhood = neighbour_solution_result.copy()
                if neighbour_cost < best_cost:
                    best_cost = neighbour_cost
                    best_solution = neighbour_solution_result.copy()
                else:
                    best_cost = best_cost
                    best_solution = best_solution
            else:
                best_cost_neighbourhood = best_cost_neighbourhood
                best_solution_neighbourhood = best_solution_neighbourhood

    return best_cost, best_solution

In [22]:
solution_before_NS5 = [4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18, 4, 4, 18, 18, 12, 18, 4, 4, 4, 18, 18]

print ('Solution before NS5', solution_before_NS5)

solution_after_NS5 = NS5 (solution_before_NS5, CAB_25_nodes_flow, CAB_25_nodes_cost)

print ('Solution after NS5', solution_after_NS5)

Solution before NS5 [4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18, 4, 4, 18, 18, 12, 18, 4, 4, 4, 18, 18]
Solution after NS5 (8591219259.912952, [4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18, 4, 4, 18, 18, 12, 18, 4, 12, 4, 18, 18])


In [23]:
# Test Chosen Neighbourhood Structure - Candidate Solution based on Random NS

operation_list = [NS1, NS2, NS3]
random_NS = random.choice (operation_list)

candidate_solution_test1 = random_NS (test_solution_1, CAB_25_nodes_cost)

print ("Random NS: ", random_NS)
print ("Candidate Solution: ", candidate_solution_test1)

Random NS:  <function NS2 at 0x11cdb8670>
Candidate Solution:  [4, 5, 5, 4, 5, 5, 10, 10, 5, 10]


## Simulated Annealing Algorithm

In Simulated Annealing, there are several things to be considered, which are:
- Temperature Function
- Initial Temperature
- The Rate of Temperature Change
- Temperature Change Strategy (Non Adaptive, Adaptive, Constant)
- Stopping Criteria

In [24]:
# Simulated Annealing Alogrithm based on Cooling Schedule Type 2

def simulated_annealing_cooling_schedule_type2 (flow_matrix, cost_matrix, n_hubs, iter, initial_temperature):
  
  start = time.time() # Calculate Total Time
  
  initial_solution = random_initial_solution (cost_matrix, n_hubs) # Initial Solution Function as defined before
  initial_cost = network_cost (initial_solution, flow_matrix, cost_matrix) # Initial Cost Function as defined before
  
  # Current Solution and Cost
  current_solution = initial_solution.copy() # Current Solution = Initial Solution
  current_cost = initial_cost # Current Cost = Initial Cost
  
  # Best Solution and Cost
  best_solution = initial_solution.copy() # Best Solution = Initial Solution
  best_cost = initial_cost # Best Cost = Initial Cost
  
  # Parameter
  K = iter # K equals with iteration number (Max Iteration). Must be defined by researchers. More iteration then more accurate.

  current_temperature = initial_temperature # T
  j = 1 # current iteration
  total_iter_executed = 0
  while j < K:
    total_iter_executed +=1
    freeze_temperature = 0
    
    # Creating Candidate Solution and Candidate Cost based on random neighbourhood

    operator_selected = random.choice ([NS1, NS5])
    if operator_selected == NS1:
      candidate_solution = NS1 (current_solution, cost_matrix)
      candidate_cost = network_cost (candidate_solution, flow_matrix, cost_matrix)
    else:
      local_search = NS5 (current_solution, flow_matrix, cost_matrix)
      candidate_solution = local_search[1]
      candidate_cost = local_search[0]

    # Calculating Delta Cost
    delta_cost = candidate_cost - current_cost

    # If candidate cost is lower than current cost, then accept.
    if candidate_cost < current_cost:
        current_solution = candidate_solution.copy()
        current_cost = candidate_cost
        if candidate_cost < best_cost:
          best_solution = candidate_solution.copy()
          best_cost = candidate_cost
        else:
          best_solution = best_solution
          best_cost = best_cost
    else: 
        random_number = random.random ()
        
        p_transition = min (1, math.exp(-delta_cost/current_temperature))
        
        if p_transition > random_number:
          current_solution = candidate_solution.copy()
          current_cost = candidate_cost
        else:
          current_solution = current_solution
          current_cost = current_cost

    if current_temperature > freeze_temperature:
        cooling_rate_alpha = 0.95
        current_temperature = cooling_rate_alpha * current_temperature
    else:
        break

    j += 1
    
    if j == K:
      break

  end = time.time()
  total_time = end - start

  return best_cost, best_solution, total_time, total_time/iter

In [25]:
# Simulate Simulated Annealing - CAB 10 dataset - 3 Hubs
total_running = 10
minimum_value_cab10_3_sa = []

for i in range(total_running):
    random.seed(i)
    result = simulated_annealing_cooling_schedule_type2 (CAB_10_nodes_flow, CAB_10_nodes_cost, 3, 250, 20000)
    minimum_value_cab10_3_sa.append(result)

minimum_value_dataframe_cab10_3_sa = pd.DataFrame (minimum_value_cab10_3_sa, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab10_3_sa

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.143905e+08,"[9, 9, 9, 9, 9, 9, 7, 7, 9, 10]",1.403801,0.005615
1,8.377146e+08,"[4, 6, 6, 4, 4, 6, 4, 4, 9, 4]",1.575611,0.006302
2,8.280907e+08,"[5, 6, 6, 4, 5, 6, 5, 4, 6, 5]",1.488146,0.005953
3,8.377146e+08,"[4, 6, 6, 4, 4, 6, 4, 4, 9, 4]",1.628366,0.006513
4,8.280907e+08,"[5, 6, 6, 4, 5, 6, 5, 4, 6, 5]",1.631697,0.006527
5,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",1.600764,0.006403
6,8.143905e+08,"[9, 9, 9, 9, 9, 9, 7, 7, 9, 10]",1.519305,0.006077
7,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",1.591485,0.006366
8,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",1.771357,0.007085
9,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",1.474884,0.005900


In [337]:
minimum_value_dataframe_cab10_3_sa.to_csv ('minimum_value_dataframe_cab10_3_sa.csv')

In [323]:
# Simulate Simulated Annealing - CAB 25 dataset - 3 Hubs
total_running = 10
minimum_value_cab25_3_sa_type2 = []

for i in range(total_running):
    random.seed(i)
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 3, 250, 20000)
    minimum_value_cab25_3_sa_type2.append(result)

minimum_value_dataframe_cab25_3_sa_type2 = pd.DataFrame (minimum_value_cab25_3_sa_type2, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_sa_type2

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,9.785705e+09,"[21, 17, 17, 21, 21, 17, 21, 21, 21, 21, 21, 2...",24.190183,0.096761
1,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",29.524411,0.118098
2,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",28.433244,0.113733
3,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",30.056802,0.120227
4,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",30.205297,0.120821
5,9.823429e+09,"[1, 17, 17, 21, 21, 17, 21, 21, 21, 21, 21, 21...",29.298882,0.117196
6,8.541480e+09,"[5, 18, 18, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",29.522864,0.118091
7,9.246816e+09,"[1, 20, 20, 20, 20, 20, 1, 19, 20, 1, 1, 19, 1...",26.660529,0.106642
8,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",31.029028,0.124116
9,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",25.385976,0.101544


In [338]:
minimum_value_dataframe_cab25_3_sa_type2.to_csv ('minimum_value_dataframe_cab25_3_sa_type2.csv')

In [327]:
# Simulate Simulated Annealing - CAB 25 dataset - 5 Hubs
total_running = 10
minimum_value_cab25_5_sa_type2 = []

for i in range(total_running):
    random.seed(i)
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 5, 250, 20000)
    minimum_value_cab25_5_sa_type2.append(result)

minimum_value_dataframe_cab25_5_sa_type2 = pd.DataFrame (minimum_value_cab25_5_sa_type2, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_sa_type2

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,9.109520e+09,"[9, 25, 17, 9, 9, 9, 11, 11, 9, 11, 11, 11, 11...",48.617084,0.194468
1,8.392631e+09,"[21, 18, 3, 21, 6, 6, 21, 21, 6, 21, 21, 12, 2...",57.540363,0.230161
2,8.199697e+09,"[21, 17, 3, 21, 21, 17, 21, 21, 21, 21, 21, 12...",56.848030,0.227392
3,8.484572e+09,"[5, 18, 18, 5, 5, 5, 5, 8, 5, 5, 5, 12, 5, 5, ...",57.948509,0.231794
4,7.949086e+09,"[21, 18, 18, 21, 21, 18, 7, 21, 18, 7, 21, 12,...",61.976652,0.247907
5,7.834005e+09,"[4, 17, 17, 4, 4, 4, 4, 8, 4, 4, 4, 12, 4, 24,...",55.358237,0.221433
6,8.290712e+09,"[1, 18, 3, 4, 4, 4, 1, 19, 4, 1, 4, 19, 1, 1, ...",58.101556,0.232406
7,8.082029e+09,"[13, 18, 18, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",55.302447,0.221210
8,8.672510e+09,"[1, 6, 6, 6, 6, 6, 7, 8, 6, 7, 7, 12, 1, 1, 6,...",64.237400,0.256950
9,7.822886e+09,"[1, 17, 17, 4, 4, 6, 4, 4, 6, 1, 4, 12, 1, 1, ...",53.217943,0.212872


In [339]:
minimum_value_dataframe_cab25_5_sa_type2.to_csv ('minimum_value_dataframe_cab25_5_sa_type2.csv')

In [331]:
# Simulate Simulated Annealing - TR 55 dataset - 3 Hubs
total_running = 10
minimum_value_tr55_3_sa_type2 = []

for i in range(total_running):
    random.seed(i+100)
    result = simulated_annealing_cooling_schedule_type2 (TR_55_nodes_flow, TR_55_nodes_cost, 3, 150, 30000)
    minimum_value_tr55_3_sa_type2.append(result)

minimum_value_dataframe_tr55_3_sa_type2 = pd.DataFrame (minimum_value_tr55_3_sa_type2, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr55_3_sa_type2

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,3.083033e+10,"[1, 1, 41, 30, 30, 41, 30, 30, 41, 41, 30, 30,...",172.508945,1.150060
1,3.166239e+10,"[55, 55, 15, 32, 32, 15, 32, 32, 15, 15, 32, 3...",147.732284,0.984882
2,2.946977e+10,"[45, 17, 17, 45, 45, 17, 30, 30, 17, 17, 45, 3...",172.736904,1.151579
3,3.136812e+10,"[25, 25, 48, 30, 25, 48, 30, 30, 48, 48, 30, 3...",212.818554,1.418790
4,3.045621e+10,"[19, 19, 19, 31, 31, 19, 26, 26, 19, 19, 31, 2...",151.413947,1.009426
5,2.972811e+10,"[19, 19, 43, 43, 30, 43, 30, 30, 19, 19, 30, 3...",192.981291,1.286542
6,3.205186e+10,"[55, 55, 46, 18, 18, 46, 18, 18, 46, 46, 18, 1...",177.654615,1.184364
7,2.973179e+10,"[4, 15, 15, 4, 4, 15, 30, 30, 15, 15, 4, 30, 3...",196.284914,1.308566
8,3.044710e+10,"[55, 55, 55, 31, 31, 55, 26, 26, 55, 55, 31, 2...",163.660288,1.091069
9,2.999615e+10,"[19, 19, 19, 18, 18, 19, 18, 18, 19, 19, 18, 1...",179.966641,1.199778


In [340]:
minimum_value_dataframe_tr55_3_sa_type2.to_csv ('minimum_value_dataframe_tr55_3_sa_type2.csv')

In [332]:
# Simulate Simulated Annealing - TR 55 dataset - 5 Hubs
total_running = 10
minimum_value_tr55_5_sa_type2 = []

for i in range(total_running):
    random.seed(i+100)
    result = simulated_annealing_cooling_schedule_type2 (TR_55_nodes_flow, TR_55_nodes_cost, 5, 150, 30000)
    minimum_value_tr55_5_sa_type2.append(result)

minimum_value_dataframe_tr55_5_sa_type2 = pd.DataFrame (minimum_value_tr55_5_sa_type2, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr55_5_sa_type2

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,2.419478e+10,"[55, 55, 15, 4, 33, 15, 33, 30, 15, 15, 4, 30,...",351.686729,2.344578
1,3.603978e+10,"[47, 42, 9, 47, 47, 9, 33, 33, 9, 9, 33, 33, 3...",356.733367,2.378222
2,3.017087e+10,"[25, 25, 48, 18, 31, 48, 18, 18, 48, 48, 18, 1...",341.113970,2.274093
3,2.696746e+10,"[45, 42, 46, 4, 45, 46, 30, 30, 46, 42, 4, 30,...",452.925061,3.019500
4,3.068988e+10,"[47, 19, 15, 18, 18, 15, 18, 18, 15, 15, 18, 1...",295.285397,1.968569
5,2.847677e+10,"[1, 1, 15, 4, 45, 15, 7, 4, 15, 15, 4, 7, 7, 4...",371.592033,2.477280
6,3.359806e+10,"[34, 34, 21, 52, 52, 21, 52, 52, 15, 15, 52, 5...",352.042464,2.346950
7,2.553873e+10,"[45, 15, 15, 4, 32, 15, 32, 32, 15, 15, 32, 32...",393.301002,2.622007
8,2.696746e+10,"[45, 42, 46, 4, 45, 46, 30, 30, 46, 42, 4, 30,...",353.920055,2.359467
9,2.773683e+10,"[29, 15, 36, 29, 33, 36, 33, 30, 15, 36, 33, 3...",385.033314,2.566889


In [341]:
minimum_value_dataframe_tr55_5_sa_type2.to_csv ('minimum_value_dataframe_tr55_5_sa_type2.csv')

In [334]:
# Simulate Simulated Annealing - TR 81 dataset - 5 Hubs
total_running = 10
minimum_value_tr81_5_sa_type2 = []

for i in range(total_running):
    random.seed(i+100)
    result = simulated_annealing_cooling_schedule_type2 (TR_81_nodes_flow, TR_81_nodes_cost, 5, 150, 50000)
    minimum_value_tr81_5_sa_type2.append(result)

minimum_value_dataframe_tr81_5_sa_type2 = pd.DataFrame (minimum_value_tr81_5_sa_type2, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr81_5_sa_type2

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,4.660497e+10,"[51, 23, 45, 23, 19, 19, 45, 23, 45, 45, 34, 2...",1246.909564,8.312730
1,4.847113e+10,"[27, 27, 43, 25, 60, 6, 43, 25, 43, 43, 43, 25...",1139.516117,7.596774
2,4.853979e+10,"[46, 46, 11, 44, 60, 71, 11, 60, 11, 11, 11, 4...",1067.472518,7.116483
3,5.316036e+10,"[40, 40, 16, 28, 40, 40, 45, 28, 45, 16, 16, 2...",1487.867946,9.919120
4,4.998432e+10,"[80, 80, 3, 58, 58, 26, 3, 58, 3, 16, 26, 58, ...",915.293540,6.101957
5,4.768465e+10,"[1, 1, 64, 12, 14, 14, 64, 28, 64, 64, 14, 12,...",1257.812100,8.385414
6,4.973995e+10,"[68, 23, 41, 25, 68, 68, 68, 61, 41, 41, 41, 2...",1134.077985,7.560520
7,4.999169e+10,"[1, 1, 54, 62, 5, 54, 54, 61, 54, 54, 54, 62, ...",1249.774553,8.331830
8,4.618310e+10,"[46, 46, 26, 52, 6, 6, 26, 52, 26, 41, 26, 46,...",1093.485677,7.289905
9,4.819869e+10,"[38, 21, 6, 25, 38, 6, 6, 25, 6, 54, 54, 21, 2...",1123.281642,7.488544


In [342]:
minimum_value_dataframe_tr81_5_sa_type2.to_csv ('minimum_value_dataframe_tr81_5_sa_type2.csv')

In [335]:
# Simulate Simulated Annealing - TR 81 dataset - 5 Hubs
total_running = 10
minimum_value_tr81_7_sa_type2 = []

for i in range(total_running):
    random.seed(i+100)
    result = simulated_annealing_cooling_schedule_type2 (TR_81_nodes_flow, TR_81_nodes_cost, 7, 150, 50000)
    minimum_value_tr81_7_sa_type2.append(result)

minimum_value_dataframe_tr81_7_sa_type2 = pd.DataFrame (minimum_value_tr81_7_sa_type2, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr81_7_sa_type2

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,4.267079e+10,"[80, 80, 68, 13, 19, 68, 68, 23, 45, 45, 41, 2...",2025.763260,13.505088
1,4.680196e+10,"[46, 46, 3, 25, 60, 6, 3, 25, 3, 3, 3, 21, 21,...",1740.889843,11.605932
2,4.588010e+10,"[46, 46, 54, 24, 19, 71, 50, 24, 54, 54, 54, 2...",1871.691579,12.477944
3,4.694510e+10,"[38, 38, 16, 49, 40, 40, 16, 28, 10, 10, 16, 4...",2258.740657,15.058271
4,4.302164e+10,"[1, 1, 3, 24, 6, 6, 3, 24, 3, 16, 16, 24, 24, ...",1530.512613,10.203417
5,4.536797e+10,"[1, 1, 42, 76, 71, 71, 42, 8, 45, 45, 54, 1, 7...",1976.082838,13.173886
6,4.407034e+10,"[80, 80, 3, 62, 68, 68, 3, 61, 3, 3, 3, 62, 21...",1936.244671,12.908298
7,4.764781e+10,"[80, 80, 43, 4, 55, 6, 43, 4, 43, 43, 43, 21, ...",2120.218520,14.134790
8,4.044057e+10,"[80, 80, 26, 23, 6, 6, 26, 28, 45, 45, 26, 23,...",1935.630889,12.904206
9,4.382008e+10,"[1, 1, 71, 25, 71, 71, 9, 25, 9, 9, 34, 25, 13...",1906.840090,12.712267


In [343]:
minimum_value_dataframe_tr81_7_sa_type2.to_csv ('minimum_value_dataframe_tr81_7_sa_type2.csv')

In [357]:
# Simulate Simulated Annealing - RGP 100 dataset - 10 Hubs
total_running = 10
minimum_value_rgp100_7_sa_type2 = []

for i in range(total_running):
    random.seed(i+100)
    result = simulated_annealing_cooling_schedule_type2 (RGP_100_nodes_flow, RGP_100_nodes_cost, 7, 30, 80000)
    minimum_value_rgp100_7_sa_type2.append(result)

minimum_value_dataframe_rgp100_7_sa_type2 = pd.DataFrame (minimum_value_rgp100_7_sa_type2, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_rgp100_7_sa_type2

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,1.448807e+11,"[91, 91, 19, 91, 19, 19, 99, 51, 19, 99, 99, 9...",831.185669,27.706189
1,1.440307e+11,"[60, 75, 85, 46, 60, 85, 7, 75, 7, 85, 75, 70,...",731.314726,24.377158
2,1.438732e+11,"[19, 87, 19, 22, 19, 92, 46, 46, 19, 22, 46, 7...",835.639867,27.854662
3,1.473442e+11,"[28, 89, 59, 95, 90, 92, 28, 90, 92, 89, 90, 9...",731.364804,24.378827
4,1.445256e+11,"[26, 94, 3, 43, 51, 3, 99, 51, 3, 3, 99, 99, 9...",626.045094,20.868170
5,1.441996e+11,"[1, 1, 75, 45, 68, 1, 76, 75, 40, 1, 75, 45, 1...",732.680138,24.422671
6,1.422113e+11,"[91, 91, 3, 91, 68, 8, 91, 8, 3, 3, 3, 3, 91, ...",831.486072,27.716202
7,1.458667e+11,"[80, 80, 80, 55, 85, 85, 99, 75, 32, 80, 75, 9...",732.546654,24.418222
8,1.440351e+11,"[92, 17, 36, 36, 36, 92, 85, 36, 17, 36, 11, 1...",624.851612,20.828387
9,1.429645e+11,"[58, 36, 36, 58, 36, 8, 58, 8, 58, 13, 36, 36,...",827.648836,27.588295


In [359]:
minimum_value_dataframe_rgp100_7_sa_type2.to_csv ('minimum_value_dataframe_rgp100_7_sa_type2.csv')

In [355]:
# Simulate Simulated Annealing - RGP 100 dataset - 10 Hubs
total_running = 10
minimum_value_rgp100_10_sa_type2 = []

for i in range(total_running):
    random.seed(i+100)
    result = simulated_annealing_cooling_schedule_type2 (RGP_100_nodes_flow, RGP_100_nodes_cost, 10, 30, 80000)
    minimum_value_rgp100_10_sa_type2.append(result)

minimum_value_dataframe_rgp100_10_sa_type2 = pd.DataFrame (minimum_value_rgp100_10_sa_type2, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_rgp100_10_sa_type2

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,1.427407e+11,"[91, 65, 56, 56, 19, 65, 99, 51, 19, 99, 56, 9...",1506.908006,50.230267
1,1.427568e+11,"[28, 65, 85, 78, 60, 85, 7, 75, 60, 85, 46, 70...",1148.291834,38.276394
2,1.423334e+11,"[19, 24, 19, 22, 19, 92, 46, 24, 19, 22, 24, 7...",1583.986680,52.799556
3,1.449449e+11,"[28, 10, 59, 78, 16, 92, 28, 16, 10, 10, 90, 8...",1497.833771,49.927792
4,1.414511e+11,"[1, 1, 3, 43, 51, 43, 1, 51, 17, 3, 17, 99, 94...",799.448006,26.648267
5,1.415158e+11,"[1, 1, 75, 76, 74, 43, 76, 75, 43, 43, 75, 99,...",1239.226116,41.307537
6,1.408145e+11,"[91, 91, 3, 91, 68, 8, 91, 8, 9, 3, 3, 34, 91,...",1590.691705,53.023057
7,1.436225e+11,"[80, 80, 80, 55, 85, 85, 85, 75, 32, 80, 75, 9...",1326.362838,44.212095
8,1.413354e+11,"[92, 17, 36, 36, 36, 49, 85, 36, 17, 36, 11, 1...",1147.391723,38.246391
9,1.422366e+11,"[58, 65, 36, 36, 64, 8, 64, 8, 64, 13, 36, 64,...",1058.655956,35.288532


In [356]:
minimum_value_dataframe_rgp100_10_sa_type2.to_csv ('minimum_value_dataframe_rgp100_10_sa_type2.csv')

# Sensitivity Analysis

## Sensitivity Analysis Initial Temperature CAB25 - 3 Hubs

In [371]:
# Simulate Simulated Annealing - CAB 25 dataset - 3 Hubs - Initial Temp 10000
total_running = 10
minimum_value_cab25_3_sa_type2_initial_temp_10000 = []

for i in range(total_running):
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 3, 250, 10000)
    minimum_value_cab25_3_sa_type2_initial_temp_10000.append(result)

minimum_value_dataframe_cab25_3_sa_type2_initial_temp_10000 = pd.DataFrame (minimum_value_cab25_3_sa_type2_initial_temp_10000, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_sa_type2_initial_temp_10000

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",29.850913,0.119404
1,9.693903e+09,"[20, 20, 17, 20, 20, 20, 11, 11, 20, 11, 11, 1...",30.182056,0.120728
2,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",24.011344,0.096045
3,9.815060e+09,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 22, 5, 14, 5...",27.180839,0.108723
4,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",29.520075,0.118080
5,9.785705e+09,"[21, 17, 17, 21, 21, 17, 21, 21, 21, 21, 21, 2...",31.611779,0.126447
6,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",26.052688,0.104211
7,8.457151e+09,"[21, 18, 18, 21, 21, 18, 21, 21, 18, 21, 21, 1...",30.978980,0.123916
8,8.714300e+09,"[5, 17, 17, 5, 5, 5, 5, 19, 5, 5, 5, 19, 5, 5,...",27.477810,0.109911
9,9.795379e+09,"[21, 18, 18, 21, 21, 18, 21, 21, 21, 21, 21, 2...",29.178278,0.116713


In [372]:
minimum_value_dataframe_cab25_3_sa_type2_initial_temp_10000.to_csv ('minimum_value_dataframe_cab25_3_sa_type2_initial_temp_10000.csv')

In [373]:
# Simulate Simulated Annealing - CAB 25 dataset - 3 Hubs - Initial Temp 20000
total_running = 10
minimum_value_cab25_3_sa_type2_initial_temp_20000 = []

for i in range(total_running):
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 3, 250, 20000)
    minimum_value_cab25_3_sa_type2_initial_temp_20000.append(result)

minimum_value_dataframe_cab25_3_sa_type2_initial_temp_20000 = pd.DataFrame (minimum_value_cab25_3_sa_type2_initial_temp_20000, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_sa_type2_initial_temp_20000

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",27.021386,0.108086
1,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",29.115139,0.116461
2,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",25.743977,0.102976
3,9.695635e+09,"[2, 2, 2, 4, 2, 2, 7, 4, 2, 7, 4, 7, 4, 2, 4, ...",30.893698,0.123575
4,8.617586e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 22, 4, 18,...",29.056928,0.116228
5,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",27.154735,0.108619
6,9.529630e+09,"[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...",30.454805,0.121819
7,8.457151e+09,"[21, 18, 18, 21, 21, 18, 21, 21, 18, 21, 21, 1...",28.414597,0.113658
8,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",26.699499,0.106798
9,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",27.766901,0.111068


In [374]:
minimum_value_dataframe_cab25_3_sa_type2_initial_temp_20000.to_csv ('minimum_value_dataframe_cab25_3_sa_type2_initial_temp_20000.csv')

In [375]:
# Simulate Simulated Annealing - CAB 25 dataset - 3 Hubs - Initial Temp 30000
total_running = 10
minimum_value_cab25_3_sa_type2_initial_temp_30000 = []

for i in range(total_running):
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 3, 250, 30000)
    minimum_value_cab25_3_sa_type2_initial_temp_30000.append(result)

minimum_value_dataframe_cab25_3_sa_type2_initial_temp_30000 = pd.DataFrame (minimum_value_cab25_3_sa_type2_initial_temp_30000, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_sa_type2_initial_temp_30000

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",25.115447,0.100462
1,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",28.437012,0.113748
2,9.609384e+09,"[6, 17, 17, 6, 6, 6, 11, 11, 6, 11, 11, 11, 11...",29.736923,0.118948
3,9.529630e+09,"[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...",26.509846,0.106039
4,1.090983e+10,"[5, 17, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,...",25.795721,0.103183
5,9.643062e+09,"[20, 20, 20, 20, 20, 20, 8, 8, 20, 8, 8, 8, 20...",26.235673,0.104943
6,9.732268e+09,"[1, 18, 18, 11, 1, 18, 11, 11, 18, 11, 11, 11,...",26.768263,0.107073
7,9.256442e+09,"[20, 20, 20, 20, 20, 20, 20, 12, 20, 20, 20, 1...",25.853654,0.103415
8,9.580185e+09,"[20, 20, 20, 20, 20, 20, 20, 8, 20, 20, 20, 12...",28.240224,0.112961
9,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",27.351139,0.109405


In [376]:
minimum_value_dataframe_cab25_3_sa_type2_initial_temp_30000.to_csv ('minimum_value_dataframe_cab25_3_sa_type2_initial_temp_30000.csv')

In [377]:
# Simulate Simulated Annealing - CAB 25 dataset - 3 Hubs - Initial Temp 50000
total_running = 10
minimum_value_cab25_3_sa_type2_initial_temp_50000 = []

for i in range(total_running):
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 3, 250, 50000)
    minimum_value_cab25_3_sa_type2_initial_temp_50000.append(result)

minimum_value_dataframe_cab25_3_sa_type2_initial_temp_50000 = pd.DataFrame (minimum_value_cab25_3_sa_type2_initial_temp_50000, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_sa_type2_initial_temp_50000

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.684333e+09,"[4, 18, 18, 4, 4, 4, 4, 19, 4, 4, 4, 19, 4, 18...",29.472857,0.117891
1,9.742914e+09,"[20, 20, 20, 20, 20, 20, 20, 12, 20, 20, 20, 1...",29.586385,0.118346
2,9.038550e+09,"[1, 20, 20, 20, 20, 20, 1, 12, 20, 1, 1, 12, 1...",30.923241,0.123693
3,9.637477e+09,"[20, 20, 20, 20, 20, 20, 8, 8, 20, 8, 8, 8, 20...",26.279717,0.105119
4,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",30.583801,0.122335
5,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",29.349917,0.117400
6,8.958818e+09,"[20, 20, 20, 20, 20, 20, 7, 7, 20, 7, 7, 12, 7...",29.977412,0.119910
7,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",29.641272,0.118565
8,1.003610e+10,"[20, 20, 20, 20, 20, 20, 7, 7, 20, 7, 7, 7, 7,...",26.109576,0.104438
9,8.478093e+09,"[21, 25, 25, 21, 21, 25, 21, 21, 25, 21, 21, 1...",27.140480,0.108562


In [378]:
minimum_value_dataframe_cab25_3_sa_type2_initial_temp_50000.to_csv ('minimum_value_dataframe_cab25_3_sa_type2_initial_temp_50000.csv')

## Sensitivity Analysis Initial Temperature CAB25 - 5 Hubs

In [379]:
# Simulate Simulated Annealing - CAB 25 dataset - 5 Hubs - Initial Temp 10000
total_running = 10
minimum_value_cab25_5_sa_type2_initial_temp_10000 = []

for i in range(total_running):
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 5, 250, 10000)
    minimum_value_cab25_5_sa_type2_initial_temp_10000.append(result)

minimum_value_dataframe_cab25_5_sa_type2_initial_temp_10000 = pd.DataFrame (minimum_value_cab25_5_sa_type2_initial_temp_10000, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_sa_type2_initial_temp_10000

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.084259e+09,"[25, 25, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 25...",57.730302,0.230921
1,7.734081e+09,"[4, 17, 17, 4, 4, 4, 10, 4, 4, 10, 4, 12, 4, 2...",51.169335,0.204677
2,8.299721e+09,"[2, 2, 2, 4, 4, 2, 4, 8, 4, 4, 4, 19, 4, 14, 4...",56.532205,0.226129
3,7.958700e+09,"[4, 2, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14, ...",63.604906,0.254420
4,8.184918e+09,"[4, 18, 18, 4, 4, 20, 10, 4, 4, 10, 4, 12, 4, ...",57.193797,0.228775
5,8.392475e+09,"[4, 18, 3, 4, 4, 4, 4, 8, 4, 4, 4, 12, 4, 18, ...",58.431082,0.233724
6,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",61.221050,0.244884
7,8.397919e+09,"[2, 2, 2, 4, 4, 2, 4, 8, 4, 4, 4, 12, 4, 2, 4,...",50.996764,0.203987
8,8.017861e+09,"[13, 17, 17, 4, 6, 6, 13, 13, 6, 13, 13, 12, 1...",55.861469,0.223446
9,7.912773e+09,"[4, 17, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",51.941802,0.207767


In [380]:
minimum_value_dataframe_cab25_5_sa_type2_initial_temp_10000.to_csv ('minimum_value_dataframe_cab25_5_sa_type2_initial_temp_10000.csv')

In [381]:
# Simulate Simulated Annealing - CAB 25 dataset - 5 Hubs - Initial Temp 20000
total_running = 10
minimum_value_cab25_5_sa_type2_initial_temp_20000 = []

for i in range(total_running):
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 5, 250, 20000)
    minimum_value_cab25_5_sa_type2_initial_temp_20000.append(result)

minimum_value_dataframe_cab25_5_sa_type2_initial_temp_20000 = pd.DataFrame (minimum_value_cab25_5_sa_type2_initial_temp_20000, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_sa_type2_initial_temp_20000

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.677901e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",54.915465,0.219662
1,7.958700e+09,"[4, 2, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14, ...",52.607478,0.210430
2,8.013446e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",60.025288,0.240101
3,8.382910e+09,"[1, 18, 18, 21, 21, 18, 21, 21, 21, 21, 21, 22...",61.742120,0.246968
4,8.187212e+09,"[2, 2, 2, 4, 4, 2, 7, 7, 4, 7, 4, 12, 4, 2, 15...",54.830208,0.219321
5,7.808925e+09,"[21, 18, 18, 4, 4, 4, 21, 21, 4, 21, 21, 12, 2...",61.253473,0.245014
6,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",61.072427,0.244290
7,9.526915e+09,"[21, 17, 3, 21, 6, 6, 21, 21, 6, 21, 21, 21, 2...",53.574819,0.214299
8,8.802455e+09,"[5, 18, 3, 4, 5, 5, 5, 8, 5, 5, 4, 8, 5, 5, 4,...",53.859903,0.215440
9,9.394334e+09,"[5, 17, 3, 4, 5, 5, 11, 11, 5, 11, 11, 11, 5, ...",55.777686,0.223111


In [382]:
minimum_value_dataframe_cab25_5_sa_type2_initial_temp_20000.to_csv ('minimum_value_dataframe_cab25_5_sa_type2_initial_temp_20000.csv')

In [383]:
# Simulate Simulated Annealing - CAB 25 dataset - 5 Hubs - Initial Temp 30000
total_running = 10
minimum_value_cab25_5_sa_type2_initial_temp_30000 = []

for i in range(total_running):
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 5, 250, 30000)
    minimum_value_cab25_5_sa_type2_initial_temp_30000.append(result)

minimum_value_dataframe_cab25_5_sa_type2_initial_temp_30000 = pd.DataFrame (minimum_value_cab25_5_sa_type2_initial_temp_30000, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_sa_type2_initial_temp_30000

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.137464e+09,"[2, 2, 2, 4, 4, 2, 7, 7, 4, 7, 4, 12, 4, 2, 4,...",57.742858,0.230971
1,8.408063e+09,"[4, 2, 17, 4, 4, 4, 4, 8, 4, 4, 4, 8, 4, 24, 4...",51.267735,0.205071
2,8.189646e+09,"[4, 17, 17, 4, 4, 4, 4, 19, 4, 4, 4, 19, 4, 14...",49.470071,0.197880
3,8.524628e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",54.732531,0.218930
4,8.895236e+09,"[20, 17, 17, 4, 20, 20, 7, 7, 20, 7, 4, 7, 4, ...",51.499989,0.206000
5,9.078633e+09,"[1, 17, 17, 9, 9, 9, 11, 11, 9, 11, 11, 11, 1,...",56.835193,0.227341
6,8.260568e+09,"[21, 2, 2, 4, 4, 2, 21, 21, 4, 21, 21, 12, 21,...",58.849463,0.235398
7,7.922192e+09,"[1, 18, 18, 21, 21, 18, 21, 21, 18, 21, 21, 12...",58.043687,0.232175
8,8.133150e+09,"[1, 17, 17, 4, 4, 4, 13, 8, 4, 13, 4, 8, 13, 1...",58.610597,0.234442
9,8.569266e+09,"[21, 18, 18, 21, 6, 6, 21, 21, 6, 21, 21, 12, ...",59.129886,0.236520


In [386]:
minimum_value_dataframe_cab25_5_sa_type2_initial_temp_30000.to_csv ('minimum_value_dataframe_cab25_5_sa_type2_initial_temp_30000.csv')

In [384]:
# Simulate Simulated Annealing - CAB 25 dataset - 5 Hubs - Initial Temp 50000
total_running = 10
minimum_value_cab25_5_sa_type2_initial_temp_50000 = []

for i in range(total_running):
    result = simulated_annealing_cooling_schedule_type2 (CAB_25_nodes_flow, CAB_25_nodes_cost, 5, 250, 50000)
    minimum_value_cab25_5_sa_type2_initial_temp_50000.append(result)

minimum_value_dataframe_cab25_5_sa_type2_initial_temp_50000 = pd.DataFrame (minimum_value_cab25_5_sa_type2_initial_temp_50000, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_sa_type2_initial_temp_50000

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.340154e+09,"[25, 25, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 25...",54.340352,0.217361
1,7.900000e+09,"[6, 17, 17, 4, 6, 6, 4, 4, 6, 4, 4, 12, 4, 14,...",59.309174,0.237237
2,8.297702e+09,"[21, 17, 17, 21, 21, 17, 21, 21, 21, 21, 21, 2...",58.432799,0.233731
3,8.213867e+09,"[4, 17, 17, 4, 4, 4, 4, 19, 4, 4, 4, 19, 4, 14...",60.937576,0.243750
4,8.390924e+09,"[1, 17, 3, 4, 4, 4, 1, 8, 4, 1, 4, 8, 1, 1, 4,...",54.276962,0.217108
5,8.253244e+09,"[5, 17, 17, 5, 5, 5, 7, 7, 5, 7, 5, 12, 5, 5, ...",57.515725,0.230063
6,8.137464e+09,"[2, 2, 2, 4, 4, 2, 7, 7, 4, 7, 4, 12, 4, 2, 4,...",52.502826,0.210011
7,8.486642e+09,"[5, 18, 18, 4, 5, 5, 5, 4, 5, 5, 4, 12, 5, 5, ...",65.140297,0.260561
8,7.862356e+09,"[1, 17, 17, 4, 5, 4, 4, 4, 4, 1, 4, 12, 1, 1, ...",51.957705,0.207831
9,8.282897e+09,"[21, 18, 18, 21, 6, 6, 21, 21, 6, 21, 21, 12, ...",54.092774,0.216371


In [385]:
minimum_value_dataframe_cab25_5_sa_type2_initial_temp_50000.to_csv ('minimum_value_dataframe_cab25_5_sa_type2_initial_temp_50000.csv')